# P4_CHANG_Nicolas

## 1 - Loading CSV file with the parameters

In [8]:
import numpy as np
import pandas as pd

# Read the CSV file into a DataFrame
file_path = 'transportation_costs.csv'
df = pd.read_csv(file_path, index_col=0)

# Extract the 'Supply' column (excluding the 'Demand' row)
supply = pd.to_numeric(df['Supply'][:-1].to_numpy())  # Use .loc for column and slice

# Extract the 'Demand' row (excluding the 'Supply' column)
demand = pd.to_numeric(df.loc['Demand'].drop('Supply').to_numpy())  # Use .loc for rows and drop 'Supply'

# Extract the cost matrix (excluding the last row and column)
cost_matrix = df.iloc[:-1, :-1].to_numpy()  # .iloc is fine for position-based indexing


## 2 - Find initial feasible solution

### a. Northwest corner rule

In [9]:
# Copy the supply and demand arrays to avoid modifying the originals
supply_copy = supply.copy()
demand_copy = demand.copy()

# Initialize the allocation matrix (same shape as the cost matrix)
allocation = np.zeros_like(cost_matrix)

# Initialize pointers for supply and demand (start from S1 and D1)
i, j = 0, 0  # Start from S1 and D1
while i < len(supply_copy) and j < len(demand_copy):
    # Determine the amount to transport (min of supply and demand)
    allocate = min(supply_copy[i], demand_copy[j])
    
    # Allocate the amount to the current position in the allocation matrix
    allocation[i, j] = allocate
    
    # Update the supply and demand
    supply_copy[i] -= allocate
    demand_copy[j] -= allocate
    
    # Move to the next cell:
    # If the supply at source i is exhausted, move to the next supplier (i)
    # If the demand at destination j is satisfied, move to the next destination (j)
    if supply_copy[i] == 0:
        i += 1  # Move to the next source
    elif demand_copy[j] == 0:
        j += 1  # Move to the next destination

# The final allocation matrix (converted to DataFrame for easier visualization)
allocation_nwcr = pd.DataFrame(allocation, index=df.index[:-1], columns=df.columns[:-1])

# Print the allocation matrix
print(allocation_nwcr)

# Calculate the total transportation cost
total_cost_nwcr = np.sum(allocation * cost_matrix)
print(f"Total transportation cost (NWCR): {total_cost_nwcr}")

     D1   D2   D3   D4
S1  400  100    0    0
S2    0  700    0    0
S3    0  100  200  500
Total transportation cost (NWCR): 14200


### b. Minimum cost method

In [10]:
# Convert the cost matrix to float to accept np.inf
cost_matrix = cost_matrix.astype(float)

# Create a copy of the cost matrix to avoid modifying it directly
cost_matrix_copy = cost_matrix.copy()

# Initialize the allocation matrix
allocation = np.zeros_like(cost_matrix)

# Copy the supply and demand to manipulate them
supply_copy = supply.copy()
demand_copy = demand.copy()

# While there is supply and demand to satisfy
while np.sum(supply_copy) > 0 and np.sum(demand_copy) > 0:
    # Find the index of the minimum cost
    i, j = np.unravel_index(np.argmin(cost_matrix_copy), cost_matrix_copy.shape)

    # Calculate the amount to allocate (min of supply and demand)
    allocate = min(supply_copy[i], demand_copy[j])

    # Allocate the amount to the allocation matrix
    allocation[i, j] = allocate

    # Update the supply and demand
    supply_copy[i] -= allocate
    demand_copy[j] -= allocate

    # If the supply is satisfied, remove this source
    if supply_copy[i] == 0:
        cost_matrix_copy[i, :] = np.inf  # Prevent this row from being selected

    # If the demand is satisfied, remove this destination
    if demand_copy[j] == 0:
        cost_matrix_copy[:, j] = np.inf  # Prevent this column from being selected

# Create the DataFrame for the allocation
allocation_mcm = pd.DataFrame(allocation, index=df.index[:-1], columns=df.columns[:-1])

# Print the allocation matrix
print(allocation_mcm)

# Calculate the total transportation cost
total_cost_mcm = np.sum(allocation * cost_matrix)
print(f"Total transportation cost (MCM): {total_cost_mcm}")

       D1     D2     D3     D4
S1  300.0    0.0  200.0    0.0
S2    0.0  700.0    0.0    0.0
S3  100.0  200.0    0.0  500.0
Total transportation cost (MCM): 12000.0


### c. Minimum Row cost method

In [11]:
# Convert the cost matrix to float to accept np.inf
cost_matrix = cost_matrix.astype(float)

# Create a copy of the cost matrix to avoid modifying it directly
cost_matrix_copy = cost_matrix.copy()

# Initialize the allocation matrix
allocation = np.zeros_like(cost_matrix)

# Copy the supply and demand to manipulate them
supply_copy = supply.copy()
demand_copy = demand.copy()

# While there is supply to allocate
for i in range(cost_matrix_copy.shape[0]):  # Process each row (source)
    while supply_copy[i] > 0:  # While the source still has supply
        # Find the column with the minimum cost in the current row
        j = np.argmin(cost_matrix_copy[i, :])

        # Calculate the amount to allocate (min of supply and demand)
        allocate = min(supply_copy[i], demand_copy[j])

        # Allocate the amount to the allocation matrix
        allocation[i, j] = allocate

        # Update the supply and demand
        supply_copy[i] -= allocate
        demand_copy[j] -= allocate

        # If the demand is satisfied, remove this destination by setting cost to np.inf
        if demand_copy[j] == 0:
            cost_matrix_copy[:, j] = np.inf  # Prevent this column from being selected

        # If the supply is exhausted, exit the loop for this row
        if supply_copy[i] == 0:
            break

# Create the DataFrame for the allocation
allocation_mrcm = pd.DataFrame(allocation, index=df.index[:-1], columns=df.columns[:-1])

# Print the allocation matrix
print(allocation_mrcm)

# Calculate the total transportation cost
total_cost_mrcm = np.sum(allocation * cost_matrix)
print(f"Total transportation cost (MRCM): {total_cost_mrcm}")

       D1     D2     D3     D4
S1    0.0    0.0  200.0  300.0
S2    0.0  700.0    0.0    0.0
S3  400.0  200.0    0.0  200.0
Total transportation cost (MRCM): 12000.0


### d. Vogel's method

In [12]:
# Convert the cost matrix to float for handling updates with np.inf
cost_matrix = cost_matrix.astype(float)

# Create a copy of the cost matrix to avoid modifying it directly
cost_matrix_copy = cost_matrix.copy()

# Initialize the allocation matrix
allocation = np.zeros_like(cost_matrix)

# Copy the supply and demand to manipulate them
supply_copy = supply.copy()
demand_copy = demand.copy()

while np.sum(supply_copy) > 0 and np.sum(demand_copy) > 0:
    # Calculate penalties for rows and columns
    row_penalty = []
    col_penalty = []

    # Compute row penalties (difference between two smallest costs in each row)
    for i in range(cost_matrix_copy.shape[0]):
        row = cost_matrix_copy[i, :]
        valid_costs = row[row != np.inf]
        if len(valid_costs) > 1:
            penalty = np.partition(valid_costs, 1)[1] - np.partition(valid_costs, 0)[0]
        elif len(valid_costs) == 1:
            penalty = valid_costs[0]  # Single cost is the penalty
        else:
            penalty = -1  # Invalid row
        row_penalty.append(penalty)

    # Compute column penalties (difference between two smallest costs in each column)
    for j in range(cost_matrix_copy.shape[1]):
        col = cost_matrix_copy[:, j]
        valid_costs = col[col != np.inf]
        if len(valid_costs) > 1:
            penalty = np.partition(valid_costs, 1)[1] - np.partition(valid_costs, 0)[0]
        elif len(valid_costs) == 1:
            penalty = valid_costs[0]  # Single cost is the penalty
        else:
            penalty = -1  # Invalid column
        col_penalty.append(penalty)

    # Find the row or column with the highest penalty
    max_row_penalty = max(row_penalty)
    max_col_penalty = max(col_penalty)

    if max_row_penalty >= max_col_penalty:
        # Choose the row with the highest penalty
        i = row_penalty.index(max_row_penalty)
        # Find the column with the smallest cost in the selected row
        j = np.argmin(cost_matrix_copy[i, :])
    else:
        # Choose the column with the highest penalty
        j = col_penalty.index(max_col_penalty)
        # Find the row with the smallest cost in the selected column
        i = np.argmin(cost_matrix_copy[:, j])

    # Calculate the allocation
    allocate = min(supply_copy[i], demand_copy[j])

    # Allocate the amount in the allocation matrix
    allocation[i, j] = allocate

    # Update supply and demand
    supply_copy[i] -= allocate
    demand_copy[j] -= allocate

    # Remove the row or column from consideration by setting its costs to np.inf
    if supply_copy[i] == 0:
        cost_matrix_copy[i, :] = np.inf
    if demand_copy[j] == 0:
        cost_matrix_copy[:, j] = np.inf

# Create a DataFrame for the allocation matrix
allocation_vam = pd.DataFrame(allocation, index=df.index[:-1], columns=df.columns[:-1])

# Print the allocation matrix
print(allocation_vam)

# Calculate the total transportation cost
total_cost_vam = np.sum(allocation * cost_matrix)
print(f"Total transportation cost (VAM): {total_cost_vam}")

       D1     D2     D3     D4
S1    0.0    0.0  200.0  300.0
S2    0.0  700.0    0.0    0.0
S3  400.0  200.0    0.0  200.0
Total transportation cost (VAM): 12000.0


## 3 - Simplex Algorithm

In that context, we want to minimize the cost of transportation

In [13]:
# Step 1 - Create helper functions for the transportation simplex method
def get_possible_next_nodes(loop, not_visited):
    last_node = loop[-1]
    nodes_in_row = [n for n in not_visited if n[0] == last_node[0]]
    nodes_in_column = [n for n in not_visited if n[1] == last_node[1]]
    if len(loop) < 2:
        return nodes_in_row + nodes_in_column
    else:
        prev_node = loop[-2]
        row_move = prev_node[0] == last_node[0]
        if row_move: return nodes_in_column
        return nodes_in_row
    
def loop_pivoting(bfs, loop):
    even_cells = loop[0::2]
    odd_cells = loop[1::2]
    get_bv = lambda pos: next(v for p, v in bfs if p == pos)
    leaving_position = sorted(odd_cells, key=get_bv)[0]
    leaving_value = get_bv(leaving_position)
    
    new_bfs = []
    for p, v in [bv for bv in bfs if bv[0] != leaving_position] + [(loop[0], 0)]:
        if p in even_cells:
            v += leaving_value
        elif p in odd_cells:
            v -= leaving_value
        new_bfs.append((p, v))
        
    return new_bfs

# Function to compute the u and v values (dual variables)
def get_us_and_vs(bfs, costs):
    us = [None] * len(costs)
    vs = [None] * len(costs[0])
    us[0] = 0
    bfs_copy = bfs.copy()
    
    while len(bfs_copy) > 0:
        for index, bv in enumerate(bfs_copy):
            i, j = bv[0]
            if us[i] is None and vs[j] is None: 
                continue
            cost = costs[i][j]
            if us[i] is None:
                us[i] = cost - vs[j]
            else:
                vs[j] = cost - us[i]
            bfs_copy.pop(index)
            break
    
    return us, vs

# Function to compute the reduced costs (opportunity costs) for non-basic variables
def get_ws(bfs, costs, us, vs):
    ws = []
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            non_basic = all([p[0] != i or p[1] != j for p, v in bfs])
            if non_basic:
                ws.append(((i, j), us[i] + vs[j] - cost))
    
    return ws

# Check if there is any possibility of improving the solution
def can_be_improved(ws):
    for p, v in ws:
        if v > 0: return True
    return False

# Get the position of the entering variable with the most positive reduced cost
def get_entering_variable_position(ws):
    ws_copy = ws.copy()
    ws_copy.sort(key=lambda w: w[1])
    return ws_copy[-1][0]

# Find the loop to apply pivoting (cycling through the basic variables)
def get_loop(bv_positions, ev_position):
    def inner(loop):
        if len(loop) > 3:
            can_be_closed = len(get_possible_next_nodes(loop, [ev_position])) == 1
            if can_be_closed: 
                return loop
        
        not_visited = list(set(bv_positions) - set(loop))
        possible_next_nodes = get_possible_next_nodes(loop, not_visited)
        for next_node in possible_next_nodes:
            new_loop = inner(loop + [next_node])
            if new_loop: 
                return new_loop
    
    return inner([ev_position])

# Step 2 - Implement the transportation simplex method
def transportation_simplex_method(supply, demand, costs, initial_allocation):
    # Helper function to get the basic feasible solution (from an initial allocation)
    def get_basic_variables(initial_allocation):
        bfs = []
        for i in range(len(initial_allocation)):
            for j in range(len(initial_allocation[i])):
                if initial_allocation[i, j] > 0:
                    bfs.append(((i, j), initial_allocation[i, j]))
        return bfs

    # Extract the basic variables (initial allocation)
    bfs = get_basic_variables(initial_allocation)
    
    # Start the simplex algorithm
    def inner(bfs):
        us, vs = get_us_and_vs(bfs, costs)
        ws = get_ws(bfs, costs, us, vs)
        
        # If there is an opportunity to improve the solution
        if can_be_improved(ws):
            ev_position = get_entering_variable_position(ws)
            loop = get_loop([p for p, v in bfs], ev_position)
            return inner(loop_pivoting(bfs, loop))  # Apply the pivoting to update BFS
        return bfs

    # Perform the transportation simplex method
    bfs_optimized = inner(bfs)
    
    # Construct the optimized solution
    optimized_allocation = np.zeros_like(costs)
    for (i, j), v in bfs_optimized:
        optimized_allocation[i][j] = v
    
    # Calculate the optimized total cost
    optimized_cost = np.sum(optimized_allocation * costs)
    
    return optimized_allocation, optimized_cost


### Use simplex method
**allocation_nwcr** : using Northwest Corner Method  
**allocation_mcm** : using Minimum Cost Method  
**allocation_mrcm** : using Minimum Row Cost Method  
**allocation_vam** : using Vogel's Method  

In [19]:

# Step 3 - Apply to your existing method (use the NWCR, MCM, MRCM, or VAM allocation)

allocation = allocation_nwcr # Change this based on method of choice

initial_allocation = allocation.to_numpy()  

# Run transportation simplex method
optimized_allocation, optimized_cost = transportation_simplex_method(supply, demand, cost_matrix, initial_allocation)

# Output results
print("Optimized Allocation Matrix:")
print(pd.DataFrame(optimized_allocation, index=df.index[:-1], columns=df.columns[:-1]))
print(f"Optimized Total Transportation Cost: {optimized_cost}")

Optimized Allocation Matrix:
       D1     D2     D3     D4
S1  300.0    0.0  200.0    0.0
S2    0.0  700.0    0.0    0.0
S3  100.0  200.0    0.0  500.0
Optimized Total Transportation Cost: 12000.0
